In [1]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.66  Python-3.12.7 torch-2.5.1+cpu CPU (11th Gen Intel Core(TM) i7-11800H 2.30GHz)
Setup complete  (16 CPUs, 31.7 GB RAM, 397.7/933.7 GB disk)


In [2]:
from ultralytics import YOLO

from IPython.display import display, Image

import roboflow

roboflow.login()

rf = roboflow.Roboflow()

project = rf.workspace("powerlineobstructiondetection").project("test-kpzbb")
dataset = project.version(3).download("yolov8")

model = YOLO('/Users/bsath/Identifying-Obstruction-in-Power-Lines/YOLOv8/yolov8n')

model.train(
    data='/Users/bsath/Identifying-Obstruction-in-Power-Lines/Test-3/data.yaml',
    epochs=50,
    imgsz=416,
    batch=4,
    device = 'cpu',
    resume=True,
    amp=True
)


In [3]:
import roboflow

roboflow.login()

rf = roboflow.Roboflow()

project = rf.workspace("powerlineobstructiondetection").project("test-kpzbb")
model = project.version(3).model

You are already logged into Roboflow. To make a different login,run roboflow.login(force=True).
loading Roboflow workspace...
loading Roboflow project...


metrics = model.val(data='/Users/bsath/Identifying-Obstruction-in-Power-Lines/Test-3/data.yaml', device='cpu')
print(metrics)

In [4]:
result = model.predict('/Users/bsath/Identifying-Obstruction-in-Power-Lines/Test-3/test/images/4dfec6c47f43a61dff52_jpg.rf.30a932e904c1733f0f6aeb79659bde2c.jpg')
predictions = result.json()
print(result.json())

{'predictions': [{'x': 467.5, 'y': 320.0, 'width': 343.0, 'height': 634.0, 'confidence': 0.854458749294281, 'class': 'wires', 'class_id': 2, 'detection_id': '1da40908-6830-4723-9929-4103bfa2dea6', 'image_path': '/Users/bsath/Identifying-Obstruction-in-Power-Lines/Test-3/test/images/4dfec6c47f43a61dff52_jpg.rf.30a932e904c1733f0f6aeb79659bde2c.jpg', 'prediction_type': 'ObjectDetectionModel'}, {'x': 263.0, 'y': 460.0, 'width': 84.0, 'height': 356.0, 'confidence': 0.809465765953064, 'class': 'wires', 'class_id': 2, 'detection_id': 'b5e9f397-99d2-47d1-bac8-326ec244c654', 'image_path': '/Users/bsath/Identifying-Obstruction-in-Power-Lines/Test-3/test/images/4dfec6c47f43a61dff52_jpg.rf.30a932e904c1733f0f6aeb79659bde2c.jpg', 'prediction_type': 'ObjectDetectionModel'}], 'image': {'width': '640', 'height': '640'}}


In [5]:
import cv2

image = cv2.imread('/Users/bsath/Identifying-Obstruction-in-Power-Lines/Test-3/test/images/4dfec6c47f43a61dff52_jpg.rf.30a932e904c1733f0f6aeb79659bde2c.jpg')
image = cv2.resize(image, (640, 640))
predictions = result.json()  # Get predictions

for prediction in predictions['predictions']:
    x1 = int(prediction['x'])
    y1 = int(prediction['y'])
    width = int(prediction['width'])
    height = int(prediction['height'])
    
    # Calculate the bottom-right corner of the bounding box
    x2 = x1 + width
    y2 = y1 + height
    
    # Draw the rectangle on the image (BGR format for OpenCV)
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green box
    
    # Add text label (class and confidence)
    label = prediction['class']
    confidence = prediction['confidence']
    cv2.putText(image, f"{label} ({confidence*100:.1f}%)", 
                (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

In [6]:
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
print(image_rgb.shape)

(640, 640, 3)


In [9]:
from PIL import Image

# Convert to PIL Image for display
image_pil = Image.fromarray(image_rgb)

# Display the image
image_pil.show()